<a href="https://colab.research.google.com/github/t0r8ern1t/other_stuff/blob/main/alpha_hack1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline

In [ ]:
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [545]:
train_df = pd.read_parquet('train.parquet')
test_df = pd.read_parquet('test.parquet')

## Обработка данных

Для базовой модели отбросим отдельные таргеты и будем использовать только total_target.

In [546]:
train_df.drop(["id", "target_1", "target_2"], axis=1, inplace=True)

In [547]:
train_df['ogrn_year'] = train_df['ogrn_year'].fillna(0)
train_df.ogrn_year = train_df.ogrn_year.astype(str).astype(int)
import numpy as np

# train_df['cat_col'] = train_df.city
# train_df.cat_col = train_df.cat_col.astype("category")
# cat_columns = train_df.select_dtypes(['category']).columns
# train_df[cat_columns] = train_df[cat_columns].apply(lambda x: x.cat.codes)
# train_df.cat_col.info()


# import seaborn as sns
# sns.scatterplot(data=train_df, x = 'city', y = 'total_target')

In [548]:
cat_cols = []
for i in train_df.columns:
  if train_df.dtypes[i] == 'object':
    cat_cols.append(i)
train_df[cat_cols] = train_df[cat_cols].astype("category")
train_df[cat_cols] = train_df[cat_cols].apply(lambda x: x.cat.codes)

nulls = train_df.isnull().sum()/len(train_df)*100
for i in train_df.columns:
  if i not in cat_cols:
    median = train_df[i].median()
    train_df[i].fillna(0, inplace=True)


['channel_code', 'city', 'city_type', 'index_city_code', 'ogrn_month', 'branch_code', 'okved', 'segment']
rko_start_months
max_end_fact_fin_deals
max_end_plan_non_fin_deals
max_start_fin_deals
max_start_non_fin_deals
min_end_fact_fin_deals
min_end_plan_non_fin_deals
min_start_fin_deals
min_start_non_fin_deals
balance_amt_avg
balance_amt_max
balance_amt_min
balance_amt_day_avg
ogrn_days_end_month
ogrn_days_end_quarter
ogrn_year
ft_registration_date
max_founderpres
min_founderpres
ogrn_exist_months
sum_of_paym_2m
sum_of_paym_6m
sum_of_paym_1y
sum_a_oper_1m
cnt_a_oper_1m
sum_b_oper_1m
cnt_b_oper_1m
sum_c_oper_1m
cnt_c_oper_1m
sum_deb_d_oper_1m
cnt_deb_d_oper_1m
sum_cred_d_oper_1m
cnt_cred_d_oper_1m
sum_deb_e_oper_1m
cnt_deb_e_oper_1m
cnt_days_deb_e_oper_1m
sum_cred_e_oper_1m
cnt_cred_e_oper_1m
cnt_days_cred_e_oper_1m
sum_deb_f_oper_1m
cnt_deb_f_oper_1m
cnt_days_deb_f_oper_1m
sum_cred_f_oper_1m
cnt_cred_f_oper_1m
cnt_days_cred_f_oper_1m
sum_deb_g_oper_1m
cnt_deb_g_oper_1m
cnt_days_deb_g_op

In [549]:
features = list(train_df)

In [550]:
df_target_1 = train_df.query('total_target == 1')
df_target_0 = train_df.query('total_target == 0')
corr1 = df_target_1[features].corr().to_numpy()
corr0 = df_target_0[features].corr().to_numpy()

In [551]:
def if_corrs(x, y):
    if abs(corr0[x][y])+0.2 < abs(corr1[x][y]):
      return True
    else:
      return False

In [552]:
import seaborn as sns

additional_features = []
flag = 0
for i in range(len(features)):
  for j in range(len(features)):
    if abs(corr1[i][j]) > 0.5 and corr1[i][j] != 1:
      if if_corrs(i, j) and [features[j], features[i]] not in additional_features:
        additional_features.append([features[i], features[j]])
        print(corr1[i][j], corr0[i][j], features[i], features[j])
index = 0
for feat in additional_features:
  train_df[f'f{index}'] = train_df[f'{feat[0]}'] * train_df[f'{feat[1]}']
  index += 1

drop_features = []
for i in range(len(features)):
  flag = False
  for j in range(len(features)):
    if i == j:
      continue
    else:
      if abs(corr1[i][j]) > 0.2:
        flag = True
        break
  if not flag and features[i] != 'total_target':
    drop_features.append(features[i])
    print(features[i])
train_df.drop(drop_features, axis=1, inplace=True)

#train_df.head(10)

0.7045100197085086 0.3975638933015573 balance_amt_max sum_deb_e_oper_1m
0.6789479605026651 0.4565261824778048 balance_amt_max sum_cred_e_oper_1m
0.591498252259944 0.36355611357213835 balance_amt_max sum_deb_e_oper_3m
0.6462180633060113 0.3974775339974439 cnt_days_deb_e_oper_1m cnt_deb_g_oper_1m
0.5407021219237377 0.33807676824241023 cnt_days_cred_e_oper_1m cnt_deb_g_oper_1m
0.7416938277469337 0.04280200159362345 cnt_deb_f_oper_1m cnt_days_deb_f_oper_1m
0.619551336943256 0.043043646358353616 cnt_deb_f_oper_1m cnt_days_deb_f_oper_3m
0.7206281663357618 0.04172944695997403 cnt_days_deb_f_oper_1m cnt_deb_f_oper_3m
0.866223543075949 0.06610874390609688 cnt_cred_f_oper_1m cnt_days_cred_f_oper_1m
0.6151902569565935 0.06993786501586183 cnt_cred_f_oper_1m cnt_days_cred_f_oper_3m
0.8538503202245523 0.5460492009389615 cnt_deb_g_oper_1m cnt_days_deb_g_oper_1m
0.7572711367952858 0.5288588900504563 cnt_deb_g_oper_1m cnt_days_deb_g_oper_3m
0.8013290176760912 0.04500906151837304 cnt_deb_f_oper_3m cnt_d

In [ ]:
train_df.cnt_deb_f_oper_1m

0        -0.018435
1        -0.008488
2        -0.018435
3        -0.018435
4         0.001458
            ...   
299995   -0.018435
299996   -0.018435
299997   -0.001858
299998   -0.018435
299999    0.018035
Name: cnt_deb_f_oper_1m, Length: 300000, dtype: float64

In [ ]:
#df_target_0.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

sns.scatterplot(data=train_df,x="total_target", y="max_end_plan_non_fin_deals")

Преобразуем тип категориальных признаков

In [ ]:
cat_cols = a

In [ ]:
train_df[cat_cols] = train_df[cat_cols].astype("category")

## Разбиение на train, validation

In [553]:
X = train_df.drop("total_target", axis=1)
y = train_df.total_target
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

## Обучение базовой модели

В качестве базовой модели возьмем LGBMClassifier

In [554]:
model = LGBMClassifier(verbosity=-1, random_state=42)
model.fit(x_train, y_train)

LGBMClassifier(random_state=42, verbosity=-1)

In [555]:
y_pred = model.predict_proba(x_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.8776646528117954

In [ ]:
# import numpy as np

# city_series = test_df.city.value_counts()
# test_df.city = test_df['city'].fillna(0)
# test_df.city.apply(lambda x: np.nan if str(x).isdigit() else x)
# test_df.city.describe()
# #test_df.city = test_df.city.astype(str).astype(int)

count     100000
unique      3943
top            0
freq       38253
Name: city, dtype: int64

Качество получилось довольно неплохим, но его еще можно улучшить

## Выгрузка результатов

In [556]:
test_df.drop("id", axis=1, inplace=True)
test_df['ogrn_year'] = test_df['ogrn_year'].fillna(0)
test_df.ogrn_year = test_df.ogrn_year.astype(str).astype(int)

cat_cols = []
for i in test_df.columns:
  if test_df.dtypes[i] == 'object':
    cat_cols.append(i)
test_df[cat_cols] = test_df[cat_cols].astype("category")
test_df[cat_cols] = test_df[cat_cols].apply(lambda x: x.cat.codes)

nulls = test_df.isnull().sum()/len(test_df)*100
for i in test_df.columns:
  if i not in cat_cols:
    median = test_df[i].median()
    test_df[i].fillna(0, inplace=True)



In [557]:
index = 0
for feat in additional_features:
  test_df[f'f{index}'] = test_df[f'{feat[0]}'] * test_df[f'{feat[1]}']
  index += 1

test_df.drop(drop_features, axis=1, inplace=True)

In [558]:
test_score = model.predict_proba(test_df)[:, 1]

In [559]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [560]:
sample_submission_df.head()

,id,score
0,300000,0.5
1,300001,0.5
2,300002,0.5
3,300003,0.5
4,300004,0.5


In [561]:
sample_submission_df["score"] = test_score

In [563]:
sample_submission_df.head()

,id,score
0,300000,0.472103
1,300001,0.510754
2,300002,0.008294
3,300003,0.008400
4,300004,0.020568


In [562]:
sample_submission_df.to_csv("my_submission.csv", index=False)